In [2]:
import pandas as pd
import os
import re
import warnings
import pandas as pd
import yaml
from pandas.core.common import SettingWithCopyWarning
warnings.simplefilter(action="ignore", category=SettingWithCopyWarning)

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:80% !important; }</style>"))

pd.set_option("display.max_rows", 2000)

In [3]:
os.listdir("./data/raw/")

['.DS_Store', 'amazon', 'mint']

In [101]:

RAW_TRANSACT_SCHEMA = [
    "date",
    "original_description",
    "transaction_type",
    "amount",
    "account_name",
]

CATEGORIZED_TRANSACT_SCHEMA = RAW_TRANSACT_SCHEMA + [
    "description",
    "category",
    "subcategory",
]

TRANSACT_KEY_COLS = ["date", "original_description", "amount"]

PATH_TO_MINT_FOLDER = "./data/raw/mint/"

PATH_TO_AMAZON_FOLDER = "./data/raw/amazon/"

PATH_TO_CATEGORIZED = "./data/categorized/transactions.csv"

PATH_TO_DESCRIPTION_MAP = "./app/descriptions.yml"

PATH_TO_CATS = "./app/categories.yml"


def load_mint_trans():
    # check that there is only one file in mint folder
    mint_files = os.listdir(PATH_TO_MINT_FOLDER)
    
    if len(mint_files) == 1:
        df = pd.read_csv(PATH_TO_MINT_FOLDER + mint_files[0])
        df.columns = [to_snake(col) for col in df.columns]
        
        # remove repeated white space characters
        df['original_description'] = df['original_description'].apply(lambda txt: ' '.join(txt.split()))

        # group by all values to aggregate duplicate transactions
        group_df = df.groupby(["date", "original_description", "transaction_type", "account_name"], as_index = False).amount.sum()

        group_df["amount"] = group_df["amount"].round(2)

        return group_df[RAW_TRANSACT_SCHEMA]
        
    else:
        return None
        print("Mint folder needs attention!")


def load_raw_trans():
    # load individual sources of raw transactions
    mint_df = load_mint_trans()
    amzn_df = load_amzn_trans()
    
    #union together
    raw_trans = pd.concat([mint_df, amzn_df])

    return raw_trans
    

def load_amzn_trans():
    # check that there is only one file in mint folder
    amzn_files = os.listdir(PATH_TO_AMAZON_FOLDER)
    
    if len(amzn_files) == 1:
        df = pd.read_csv(PATH_TO_AMAZON_FOLDER + amzn_files[0])
        df.columns = [to_snake(col) for col in df.columns]
        df["original_description"] = ("AMZN: " + df["title"] + " " + df["category"]).fillna("AMZN: unknown item / return")
        df["account_name"] = "AMAZON"
        df["amount"] = df["item_total"].apply(lambda x: float(x.replace("$", "")))
        df["transaction_type"] = "debit"
        df.rename(columns = 
                  {"order_date": "date",
                  }, inplace = True)
        
        return df[RAW_TRANSACT_SCHEMA]
        
    else:
        return None
        print("Amazon folder needs attention!")


In [113]:
def load_amzn_trans():
    # check that there is only one file in mint folder
    amzn_files = os.listdir(PATH_TO_AMAZON_FOLDER)
    
    if len(amzn_files) == 1:
        df = pd.read_csv(PATH_TO_AMAZON_FOLDER + amzn_files[0])
        df.columns = [to_snake(col) for col in df.columns]
        df["original_description"] = ("AMZN: " + df["title"] + " " + df["category"]).fillna("AMZN: unknown item / return")
        df["account_name"] = "AMAZON"
        df["amount"] = df["item_total"].apply(lambda x: float(x.replace("$", "")))
        df["transaction_type"] = "debit"
        df.rename(columns = 
                  {"order_date": "date",
                  }, inplace = True)
        
        return df[RAW_TRANSACT_SCHEMA]
        
    else:
        return None
        print("Amazon folder needs attention!")
